In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import requests

In [3]:
# Base URL for FPL API
base_url = 'https://fantasy.premierleague.com/api/'

def get_fpl_data():
    """Fetch and process FPL data"""
    r = requests.get(base_url + 'bootstrap-static/').json()
    players = pd.json_normalize(r['elements'])
    teams = pd.json_normalize(r['teams'])
    positions = pd.json_normalize(r['element_types'])
    
    # Merge data
    df = players.merge(teams, left_on='team', right_on='id')
    df = df.merge(positions, left_on='element_type', right_on='id')
    
    return df

def prepare_features(df, position):
    """Prepare features for a given position"""
    position_df = df[df['singular_name'] == position]
    
    features = [
        'minutes', 'goals_scored', 'assists', 'clean_sheets', 
        'goals_conceded', 'own_goals', 'penalties_saved', 
        'penalties_missed', 'yellow_cards', 'red_cards', 
        'saves', 'bonus', 'bps', 'influence', 'creativity', 
        'threat', 'ict_index', 'form', 'points_per_game',
        'selected_by_percent', 'now_cost'
    ]
    
    X = position_df[features]
    y = position_df['total_points']
    
    return X, y

def save_data(df):
    """Save FPL data to CSV"""
    df.to_csv('fpl_data.csv', index=False)
    

def train_models(X, y):
    """Train Random Forest and SVM models"""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_pred)
    
    svm_model = SVR(kernel='rbf')
    svm_model.fit(X_train, y_train)
    svm_pred = svm_model.predict(X_test)
    svm_mse = mean_squared_error(y_test, svm_pred)
    
    return rf_model, svm_model, rf_mse, svm_mse

def predict_best_lineup(df, rf_model, svm_model, position, num_players):
    """Predict the best lineup for a given position"""
    position_df = df[df['singular_name'] == position]
    
    features = [
        'minutes', 'goals_scored', 'assists', 'clean_sheets', 
        'goals_conceded', 'own_goals', 'penalties_saved', 
        'penalties_missed', 'yellow_cards', 'red_cards', 
        'saves', 'bonus', 'bps', 'influence', 'creativity', 
        'threat', 'ict_index', 'form', 'points_per_game',
        'selected_by_percent', 'now_cost'
    ]
    
    X = position_df[features]
    
    rf_predictions = rf_model.predict(X)
    svm_predictions = svm_model.predict(X)
    
    position_df['rf_pred'] = rf_predictions
    position_df['svm_pred'] = svm_predictions
    position_df['avg_pred'] = (position_df['rf_pred'] + position_df['svm_pred']) / 2
    
    best_lineup = position_df.nlargest(num_players, 'avg_pred')
    
    return best_lineup[['web_name', 'team_name', 'avg_pred']]

def main():
    df = get_fpl_data()
    
    positions = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
    num_players = {'Goalkeeper': 1, 'Defender': 4, 'Midfielder': 4, 'Forward': 2}
    
    for position in positions:
        print(f"\nPredicting best lineup for {position}s:")
        X, y = prepare_features(df, position)
        rf_model, svm_model, rf_mse, svm_mse = train_models(X, y)
        
        print(f"Random Forest MSE: {rf_mse:.2f}")
        print(f"SVM MSE: {svm_mse:.2f}")
        
        best_lineup = predict_best_lineup(df, rf_model, svm_model, position, num_players[position])
        print(best_lineup)

if __name__ == "__main__":
    main()


Predicting best lineup for Goalkeepers:


KeyError: "['form'] not in index"